# Preparing Eviction Data for Visualization in Tableau
This code is designed to prepare court data for visualization with Tableau.

This code reorganizes eviction case data scraped from court websites into chronologically-organized counts of evictions. 

This data is organized into yearly columns and broken down by either week or month.

This code has functionality to prepare data into three formats: weekly, monthly, and cumulative monthly.

The CSVs outputted by this code should be ready for direct upload to Tableau for visualization.

In [27]:
import pandas as pd
from datetime import date, timedelta

In [63]:
class CourtData():
    '''
    This class is designed to process court data from various states to obtain counts of specific case types over time.
    Right now, this class is designed to count eviction cases.
    
    The state must be specified so that the class knows how to process the court data, since court data formatting varies
    from state to state. Right now, this class can process court data from South Carolina and Delaware.
    
    In the future, functionality may be added for additional states.
    
    This class contains seven methods. Three of them are involved in data preprocessing. Three of them are callable
    to return different data outputs. The last one supports the callable methods.
    Preprocessing
    - get_info(self, data, parameters)
        Extracts relevant information from overall court data.
        - data is the main dataset.
        - parameters is a list of strings that tell the function which column to parse and what to look for.
    - datetime_column(self, data)
    - year_columns(self, data)
    
    Callable
    - get_monthly_counts(self, renter_households)
    - get_weekly_counts(self, start_date, end_date)
    - get_cumulative(self)
    
    Support for callables
    - join_counts(self, old_table, new_table, column_name)
    '''
    def __init__(self, filename, state, date_cutoff = None):
        self.data = pd.read_csv(filename)
        self.date_cutoff = date_cutoff
        
        # Get the years included in the dataset
        self.year_ints = list(self.data.year.unique()) # what years are we working with?
        if len(self.year_ints) > 1: # if we are working with multiple years, we will want to break those
            # years into their own columns at some point.
            self.year_ints = sorted(self.year_ints)
            self.year_cols = list(map(str, self.year_ints)) # these year strings will be column names later
        
        # This section dictates how the court data will be processed depending on the state.
        # Filter only the cases we want. The process here depends on how each state organizes its court data.
        # Delaware
        if state == 'DE':
            # Filter cases in New Castle County
            # These cases are identifiable by the Justice of the Peace number, which should be 13 or 9
            self.data = self.data[self.data.apply(lambda x: x['case_description'][10:14] == 'JP13' or x['case_description'][10:13] == 'JP9', axis=1)]
            # These parameters depend on how states organize and label their data
            parameters = ['case_id=', 'case_description', '61 - JP LANDLORD TENANT']
            self.eviction_data = self.get_info(self.data, parameters)
        # South Carolina
        elif state == 'SC':
            parameters = ['case_number=', 'case_information', 'Rule to Vacate']
            self.eviction_data = self.get_info(self.data, parameters)
        
        self.case_data = self.datetime_column(self.eviction_data)
        
        self.count_data = self.case_data['date'].value_counts().rename_axis('date').reset_index(name='counts')
        self.count_data = self.count_data.sort_values(by='date')
        
        if self.date_cutoff:
            self.count_data = self.count_data.loc[(self.count_data['date'] <= self.date_cutoff)]
        
        if len(self.year_ints) > 1:
            self.count_data = self.year_columns(self.count_data)
        
        sdate = date(2020,1,1)
        edate = date(2020,12,31)
        self.date_list = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
        self.count_data['date'] = self.date_list
        self.count_data.set_index(['date'], inplace=True)
        
    def get_info(self, data, parameters):
        '''
        This method parses court data to identify eviction cases. It records Case IDs but as of 12/01/2020, these
        Case IDs are not used elsewhere in this class.
        This class could be expanded to record additional information or cut down to not even include Case IDs.
        '''
        # Select case types of interest
        case_ids = []
        years = []
        months = []
        days = []
        
        identifier = parameters[0]
        column_name = parameters[1]
        case_of_interest = parameters[2]
        delim = ','
        for index, row in data.iterrows():
            info = row[column_name]
            if case_of_interest in info:
                case_id = info[info.index(identifier) + len(identifier):]
                case_id = case_id.partition(delim)[0]
                case_ids.append(case_id)
                years.append(row['year'])
                months.append(row['month'])
                days.append(row['day'])
        
        column_names = ['case_id', 'year', 'month', 'day']
        eviction_data = pd.DataFrame(list(zip(case_ids, years, months, days)), columns=column_names)
        return eviction_data

    def datetime_column(self, data):
        '''
        This function combines the year, month, day columns into one date column.
        '''
        date_data = data[['year', 'month', 'day']]
        date_column = pd.to_datetime(date_data)
        data['date'] = date_column
        data.drop(['year', 'month', 'day'], axis=1, inplace=True)
        return data
    
    def year_columns(self, data):
        '''
        
        '''
        # Create month-day df
        sdate = date(2020,1,1)
        edate = date(2020,12,31)
        self.date_list = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
        month_day_list = [date.strftime('%m-%d') for date in self.date_list]
        month_day_df = pd.DataFrame(month_day_list, columns=['date'])
        month_day_df = month_day_df.set_index('date')
        
        # Break data into year columns
        for year_name, year_number in zip(self.year_cols, self.year_ints):
            start_date = year_name + '-01-01'
            end_date = year_name + '-12-31'
            current_year = data.loc[(data['date'] >= start_date) & (data['date'] <= end_date)]
            month_day = []
            current_year['date'] = current_year['date'].apply(lambda x: x.strftime('%m-%d'))
            current_year = current_year.set_index('date')
            month_day_df = month_day_df.join(current_year, how='left', rsuffix='counts')
            month_day_df[year_name] = month_day_df['counts'].fillna(0)
            month_day_df = month_day_df.drop(['counts'], axis=1)
        avg_column = []
        for index, row in month_day_df.iterrows():
            avg = sum([row[year] for year in self.year_cols[:-1]])/(len(self.year_cols)-1)
            avg_column.append(avg)
        month_day_df['avg'] = avg_column
        return month_day_df
    
    def get_monthly_counts(self, renter_households):
        self.count_data.index = pd.DatetimeIndex(self.count_data.index)
        
        monthly_data = self.count_data.groupby(pd.Grouper(freq='M')).sum()
        
        if len(self.year_ints) > 1:
            for column, renter_household_count in zip(self.count_data.columns, renter_households):
                monthly_data[column] = monthly_data[column].div(renter_household_count)
                monthly_data[column] = monthly_data[column] * 10000
        else:
            monthly_data['counts'] = monthly_data['counts'].div(renter_households)
            monthly_data['counts'] = monthly_data['counts'] * 10000
        
        return monthly_data

    def get_weekly_counts(self, start_date, end_date):
        # done in Tableau
        # we just need the date range, which should be some multiple of 7 days
        case_data = self.count_data.loc[(self.count_data.index >= start_date) & (self.count_data.index <= end_date)]
        return case_data
    
    def join_counts(self, old_table, new_table, column_name):
        '''
        
        '''
        old_table = old_table.join(new_table, how='left')
        old_table = old_table.drop(['dates'], axis=1)
        old_table[column_name] = old_table['counts'].fillna(0)
        old_table = old_table.drop(['counts'], axis=1)
        return old_table
    
    def get_cumulative(self):
        if len(self.year_ints) > 1:
            cumulative_data = self.count_data.copy()
            columns = self.year_cols.copy() + ['avg']
            for year in columns:
                col_name = 'cumulative_' + year
                cumulative_data[col_name] = cumulative_data[year].cumsum()
            cumulative_data = cumulative_data.drop(columns, axis=1)
        else:
            cumulative_data = self.count_data
            cumulative_data['cumulative_counts'] = cumulative_data['count'].cumsum()
            cumulative_data = cumulative_data.drop('count')
        return cumulative_data

In [77]:
new_castle = CourtData('Downloads/LSC Work/Data Visualizations/New Castle County/delaware_court_data_through_october.csv', 'DE')
charleston = CourtData('Downloads/LSC Work/Data Visualizations/Charleston County/charleston_court_data.csv', 'SC')
greenville = CourtData('Downloads/LSC Work/Data Visualizations/Greenville County/greenville_court_data.csv', 'SC')

<ipython-input-63-6f8d45c7eb29>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year['date'] = current_year['date'].apply(lambda x: x.strftime('%m-%d'))


In [78]:
new_castle_monthly = new_castle.get_monthly_counts(renter_households=[67500, 69191, 66276, 70953, 68852.2, 68480.0])
charleston_monthly = charleston.get_monthly_counts(renter_households=[61060, 63630, 60966, 60116, 61168.200000000004, 61443.0])
greenville_monthly = greenville.get_monthly_counts(renter_households=[65891, 63234, 62260, 60220, 62001.9, 62901.25])

In [80]:
new_castle_monthly.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/new_castle_monthly.csv')
charleston_monthly.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/charleston_monthly.csv')
greenville_monthly.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/greenville_monthly.csv')

In [79]:
new_castle_cumulative = new_castle.get_cumulative()
charleston_cumulative = charleston.get_cumulative()
greenville_cumulative = greenville.get_cumulative()

In [81]:
new_castle_cumulative.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/new_castle_cumulative.csv')
charleston_cumulative.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/charleston_cumulative.csv')
greenville_cumulative.to_csv('Downloads/LSC Work/Data Visualizations/Most Recent Data/greenville_cumulative.csv')

## TESTS 12/01/2020
The following cells test all functionality in this class for both Delaware and South Carolina.

In [64]:
de_test = CourtData('Downloads/LSC Work/Data Visualizations/New Castle County/delaware_court_data_through_october.csv', 'DE')

<ipython-input-63-6f8d45c7eb29>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year['date'] = current_year['date'].apply(lambda x: x.strftime('%m-%d'))


In [52]:
de_test.count_data.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-01-01,5.0,0.0,1.0,10.0,5.0,4.00
2020-01-02,0.0,0.0,19.0,46.0,30.0,16.25
2020-01-03,2.0,37.0,23.0,48.0,16.0,27.50
2020-01-04,32.0,25.0,2.0,11.0,0.0,17.50
2020-01-05,18.0,14.0,14.0,2.0,6.0,12.00


In [53]:
de_monthly = de_test.get_monthly_counts(renter_households=[67500, 69191, 66276, 70953, 68852.2, 68480.0])

In [54]:
de_monthly.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-01-31,158.666667,132.098105,150.582413,148.408101,135.943369,147.342290
2020-02-29,112.740741,127.039644,152.242139,148.830916,133.183834,135.258470
2020-03-31,114.814815,111.864260,121.461766,111.200372,59.838320,114.741530
2020-04-30,128.296296,121.114018,129.307743,124.871394,1.016670,125.839661
2020-05-31,136.444444,140.914281,142.887320,135.582710,1.742864,138.909171


In [65]:
de_weekly = de_test.get_weekly_counts(start_date='2020-02-22', end_date='2020-04-15')

Index(['2016', '2017', '2018', '2019', '2020', 'avg'], dtype='object')


In [66]:
de_weekly.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-02-22,108.0,41.0,41.0,97.0,4.0,71.75
2020-02-23,46.0,57.0,45.0,0.0,3.0,37.00
2020-02-24,31.0,47.0,3.0,9.0,53.0,22.50
2020-02-25,26.0,1.0,5.0,49.0,57.0,20.25
2020-02-26,17.0,3.0,53.0,49.0,22.0,30.50


In [67]:
de_cumulative = de_test.get_cumulative()

In [68]:
de_cumulative.head()

,cumulative_2016,cumulative_2017,cumulative_2018,cumulative_2019,cumulative_2020,cumulative_avg
date,,,,,,
2020-01-01,5.0,0.0,1.0,10.0,5.0,4.00
2020-01-02,5.0,0.0,20.0,56.0,35.0,20.25
2020-01-03,7.0,37.0,43.0,104.0,51.0,47.75
2020-01-04,39.0,62.0,45.0,115.0,51.0,65.25
2020-01-05,57.0,76.0,59.0,117.0,57.0,77.25


In [69]:
sc_test = CourtData('Downloads/LSC Work/Data Visualizations/Charleston County/charleston_court_data.csv', 'SC')

<ipython-input-63-6f8d45c7eb29>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_year['date'] = current_year['date'].apply(lambda x: x.strftime('%m-%d'))


In [70]:
sc_test.count_data.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.00
2020-01-02,0.0,0.0,41.0,27.0,28.0,17.00
2020-01-03,0.0,19.0,0.0,19.0,18.0,9.50
2020-01-04,7.0,12.0,0.0,11.0,0.0,7.50
2020-01-05,13.0,26.0,0.0,0.0,0.0,9.75


In [71]:
sc_monthly = sc_test.get_monthly_counts(renter_households=[61060, 63630, 60966, 60116, 61168.200000000004, 61443.0])

In [72]:
sc_monthly.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-01-31,243.367180,224.893918,235.705147,235.378269,210.730412,234.729750
2020-02-29,194.071405,189.847556,225.043467,200.445805,200.757910,202.219944
2020-03-31,156.731084,174.446016,184.037004,170.337348,100.215471,171.419039
2020-04-30,196.364232,157.315732,187.153495,152.039391,0.490451,173.127940
2020-05-31,210.776286,190.947666,215.693993,190.964136,51.987798,202.016503


In [73]:
sc_weekly = sc_test.get_weekly_counts(start_date='2020-02-22', end_date='2020-04-15')

Index(['2016', '2017', '2018', '2019', '2020', 'avg'], dtype='object')


In [74]:
sc_weekly.head()

,2016,2017,2018,2019,2020,avg
date,,,,,,
2020-02-22,68.0,37.0,46.0,24.0,0.0,43.75
2020-02-23,55.0,60.0,71.0,0.0,0.0,46.50
2020-02-24,24.0,20.0,0.0,0.0,44.0,11.00
2020-02-25,26.0,0.0,0.0,40.0,60.0,16.50
2020-02-26,15.0,0.0,55.0,87.0,37.0,39.25


In [75]:
sc_cumulative = sc_test.get_cumulative()

In [76]:
sc_cumulative.head()

,cumulative_2016,cumulative_2017,cumulative_2018,cumulative_2019,cumulative_2020,cumulative_avg
date,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.00
2020-01-02,0.0,0.0,41.0,27.0,28.0,17.00
2020-01-03,0.0,19.0,41.0,46.0,46.0,26.50
2020-01-04,7.0,31.0,41.0,57.0,46.0,34.00
2020-01-05,20.0,57.0,41.0,57.0,46.0,43.75
